<a href="https://colab.research.google.com/github/shintaroudlotulhanafia/TugasAkhirS1-DRL/blob/main/5_TradingViewScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install websocket_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54 kB 2.9 MB/s 


In [4]:
from websocket import create_connection
import json
import random
import string
import re
import pandas as pd
import csv
from datetime import datetime
import time

def filter_raw_message(text):
    try:
        found = re.search('"m":"(.+?)",', text).group(1)
        found2 = re.search('"p":(.+?"}"])}', text).group(1)
        print(found)
        print(found2)
        return found, found2
    except AttributeError:
        print("error")
    

def generateSession():
    stringLength=12
    letters = string.ascii_lowercase
    random_string= ''.join(random.choice(letters) for i in range(stringLength))
    return "qs_" +random_string

def generateChartSession():
    stringLength=12
    letters = string.ascii_lowercase
    random_string= ''.join(random.choice(letters) for i in range(stringLength))
    return "cs_" +random_string

def prependHeader(st):
    return "~m~" + str(len(st)) + "~m~" + st

def constructMessage(func, paramList):
    #json_mylist = json.dumps(mylist, separators=(',', ':'))
    return json.dumps({
        "m":func,
        "p":paramList
        }, separators=(',', ':'))

def createMessage(func, paramList):
    return prependHeader(constructMessage(func, paramList))

def sendRawMessage(ws, message):
    ws.send(prependHeader(message))

def sendMessage(ws, func, args):
    ws.send(createMessage(func, args))

def generate_csv(a,saham,periode,wak):
    out= re.search('"s":\[(.+?)\}\]', a).group(1)
    saham=saham[4:8]
    #periode='15'
    #fileTR='TR/P'+periode+'X/dataTR_'+saham+periode+'x.txt'
    #fileTR='TR/Coba/P'+periode+'X/dataTR_'+saham+periode+'x.txt'
    #fileTR='TRC/P'+periode+'X31/dataTR_'+saham+periode+'x.txt'   #Terakhir: 24
    #fileTR='TES/P'+periode+'/TR_'+saham+periode+'.txt'
    fileTR='/content/drive/MyDrive/TugasAkhirS1-DRL/5TradingViewScraper/5.1.DataPerStock/'+saham+'.csv'
    #fileTR='TR/P'+periode+'Coba/'+saham+periode+'x.txt'
    #fileTR='datset'+periode+'X54/dataTR_'+saham+periode+'x.txt'
    #fileTR='TR/P'+periode+'300/dataTR_'+saham+periode+'1410.txt'
    try:
      f_TR = open(fileTR, 'w').close()
    except:
      print("Sudah ada")
    
    x=out.split(',{\"')
    print(len(x))
    k=0
    for j in x:
      if k!=0:
        j='{"'+j
      if k==len(x)-1:
        j=j+'}'
      print(j)  
      pard = json.loads(j)
      spard='{"saham":"'+saham+'","indeks":"'+str(pard["i"])+'","date":"'+str(pard["v"][0])+'","open":"'+str(pard["v"][1])+'","high":"'+str(pard["v"][2])+'","low":"'+str(pard["v"][3])+'","close":"'+str(pard["v"][4])+'","vol":"'+str(pard["v"][5])+'"}\n'
      print(spard)
      k=k+1
      f_TR = open(fileTR, 'a+')
      f_TR.write(spard)
      f_TR.close()
            #input()
            
   
    """
    with open('data_file1min5.csv', mode='w', newline='') as data_file:
        employee_writer = csv.writer(data_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
        employee_writer.writerow(['index', 'date', 'open', 'high', 'low', 'close', 'volume'])
        
        for xi in x:
            xi= re.split('\[|:|,|\]', xi)
            print(xi)
            ind= int(xi[1])
            ts= datetime.fromtimestamp(float(xi[4])).strftime("%Y/%m/%d, %H:%M:%S")
            employee_writer.writerow([ind, ts, float(xi[5]), float(xi[6]), float(xi[7]), float(xi[8]), float(xi[9])])
    """        


# Initialize the headers needed for the websocket connection
headers = json.dumps({
    # 'Connection': 'upgrade',
    # 'Host': 'data.tradingview.com',
    'Origin': 'https://data.tradingview.com'
    # 'Cache-Control': 'no-cache',
    # 'Upgrade': 'websocket',
    # 'Sec-WebSocket-Extensions': 'permessage-deflate; client_max_window_bits',
    # 'Sec-WebSocket-Key': '2C08Ri6FwFQw2p4198F/TA==',
    # 'Sec-WebSocket-Version': '13',
    # 'User-Agent': 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36 Edg/83.0.478.56',
    # 'Pragma': 'no-cache',
    # 'Upgrade': 'websocket'
})

    
# Then create a connection to the tunnel
#kode=['AALI','ADMF','ADMG','AMOR','AMRT','ANDI','ARNA','ARTI','ARTO','ASGR','ASPI','AUTO','BABP','BAPI','BEEF','BELL','BGTG','BHAT','BHIT','BINA','BIPP','BIRD','BKSL','BLTA','BSIM','BTPS','CAMP','CARE','CASH','CASS','CBMF','CENT','CLEO','CMNP','COCO','CPIN','CSRA','DEAL','DIVA','DLTA','DMMX','DPNS','DSFI','DUCK','DWGL','DYAN','EMTK','FMII','FREN','GDST','GGRM','GOOD','GPRA','GSMF','HEAL','HEXA','HMSP','HOMI','HRTA','IMJS','IMPC','INAF','INDR','INDY','INTP','IPCC','IPCM','IPOL','ITIC','ITMA','JAST','JAYA','JKON','JRPT','JTPE','KAYU','KBLI','KEEN','KICI','KMTR','KPAS','KREN','LPCK','LUCK','MAIN','MAPA','MARI','MARK','MAYA','MCAS','MDIA','MEGA','MGRO','MLIA','MMLP','MPMX','MPPA','MRAT','MSIN','MTDL','MTPS','MYOR','NATO','NICK','NIKL','NISP','NOBU','OPMS','PALM','PAMG','PBID','PBRX','PLAN','PMMP','PNBN','PNBS','PNLF','POLL','PPGL','PPRE','PRDA','PRIM','PSSI','PTPW','PTRO','PTSN','PURA','PURE','PYFA','RAJA','REAL','ROTI','SAME','SBAT','SDRA','SFAN','SGER','SGRO','SHIP','SILO','SIMP','SIPD','SKRN','SMCB','SMDR','SMSM','SOTS','SPTO','SQMI','SRTG','TBLA','TDPM','TFAS','TGRA','TIRA','TMPO','TNCA','TOTL','TOYS','TPIA','TRIM','TRIN','TRIS','TRUK','TSPC','UCID','UFOE','ULTJ','UNIQ','VICI','VIVA','VRNA','WIFI','WIIM','WINS','WMUU','WOWS','XSSI','YELO','YULE','ZBRA','XXXX']
#kode=['AALI','ADMF','ADMG','AMOR','AMRT','ANDI','ARNA','ARTI','ARTO','ASGR','ASPI','AUTO','BABP','BAPI','BEEF','BELL','BGTG','BHAT','BHIT','BINA','BIPP','BIRD','BKSL','BLTA','BSIM','BTPS','CAMP','CARE','CASH','CASS','CBMF','CENT','CLEO','CMNP','COCO','CPIN','CSRA','DEAL','DIVA','DLTA','DMMX','DPNS','DSFI','DUCK','DWGL','DYAN','EMTK','FMII','FREN','GDST','GGRM','GOOD','GPRA','GSMF','HEAL','HEXA','HMSP','HOMI','HRTA','IMJS','IMPC','INAF','INDR','INDY','INTP','IPCC','IPCM','IPOL','ITIC','ITMA','JAST','JAYA','JKON','JRPT','JTPE','KAYU','KBLI','KEEN','KICI','KMTR','KPAS','KREN','LPCK','LUCK','MAIN','MAPA','MARI','MARK','MAYA','MCAS','MDIA','MEGA','MGRO','MLIA','MMLP','MPMX','MPPA','MRAT','MSIN','MTDL','MTPS','MYOR','NATO','NICK','NIKL','NISP','NOBU','OPMS','PALM','PAMG','PBID','PBRX','PLAN','PMMP','PNBN','PNBS','PNLF','POLL','PPGL','PPRE','PRDA','PRIM','PSSI','PTPW','PTRO','PTSN','PURA','PURE','PYFA','RAJA','REAL','ROTI','SAME','SBAT','SDRA','SFAN','SGER','SGRO','SHIP','SILO','SIMP','SIPD','SKRN','SMCB','SMDR','SMSM','SOTS','SPTO','SQMI','SRTG','TBLA','TDPM','TFAS','TGRA','TIRA','TMPO','TNCA','TOTL','TOYS','TPIA','TRIM','TRIN','TRIS','TRUK','TSPC','UCID','UFOE','ULTJ','UNIQ','VICI','VIVA','VRNA','WIFI','WIIM','WINS','WMUU','WOWS','XSSI','YELO','YULE','ZBRA','XXXX']
#kode=['ADRO','ABBA','ACES','ACST','ADHI','AGII','AGRO','AISA','AKRA','AMAR','ANTM','APIC','APLN','ASII','ASRI','ASSA','BBCA','BEST','BOGA','BOSS','BRIS','BRMS','BRPT','BSDE','BULL','BUKA','BBHI','BBKP','BUMI','BWPT','CTRA','DGNS','DILD','DKFT','DMAS','DOID','DSNG','ELSA','ENRG','ERAA','ESSA','EXCL','FILM','FIRE','FPNI','GIAA','GJTL','GMFI','HKMU','HOKI','HRUM','ICBP','IKAN','IMAS','INCO','INDF','INKP','INPC','IPTV','ITMG','IRRA','ISAT','ISSP','JPFA','JSKY','JSMR','KAEF','KIJA','KIOS','KLBF','KOTA','KPIG','KRAS','LINK','LPKR','LPPF','LSIP','MAPI','MCOR','MDKA','MEDC','META','MIDI','MIKA','MLPL','MNCN','PGAS','POWR','PPRO','PSAB','PTBA','PTPP','PWON','RALS','SCMA','SIDO','SMBR','SMGR','SMRA','SOCI','SRIL','SSIA','SSMS','TBIG','TCPI','TINS','TKIM','TLKM','TOWR','UNTR','UNVR','WEGE','WIKA','WOOD','WSBP','WSKT','WTON','ZINC','XXXX']
#kode=['ADRO','ABBA','ACES','ACST','ADHI','AGII','AGRO','AISA','AKRA','AMAR','ANTM','APIC','APLN','ASII','ASRI','ASSA','BBCA','BEST','BOGA','BOSS','BRIS','BRMS','BRPT','BSDE','BULL','BUKA','BBHI','BBKP','BUMI','BWPT','CTRA','DGNS','DILD','DKFT','DMAS','DOID','DSNG','ELSA','ENRG','ERAA','ESSA','EXCL','FILM','FIRE','FPNI','GIAA','GJTL','GMFI','HKMU','HOKI','HRUM','ICBP','IKAN','IMAS','INCO','INDF','INKP','INPC','IPTV','ITMG','IRRA','ISAT','ISSP','JPFA','JSKY','JSMR','KAEF','KIJA','KIOS','KLBF','KOTA','KPIG','KRAS','LINK','LPKR','LPPF','LSIP','MAPI','MCOR','MDKA','MEDC','META','MIDI','MIKA','MLPL','MNCN','PGAS','POWR','PPRO','PSAB','PTBA','PTPP','PWON','RALS','SCMA','SIDO','SMBR','SMGR','SMRA','SOCI','SRIL','SSIA','SSMS','TBIG','TCPI','TINS','TKIM','TLKM','TOWR','UNTR','UNVR','WEGE','WIKA','WOOD','WSBP','WSKT','WTON','ZINC','XXXX']
#kode=['ELSA','ENRG','ERAA','ESSA','EXCL','FILM','FIRE','FPNI','GIAA','GJTL','GMFI','HKMU','HOKI','HRUM','ICBP','IKAN','IMAS','INCO','INDF','INKP','INPC','IPTV','ITMG','IRRA','ISAT','ISSP','JPFA','JSKY','JSMR','KAEF','KIJA','KIOS','KLBF','KOTA','KPIG','KRAS','LINK','LPKR','LPPF','LSIP','MAPI','MCOR','MDKA','MEDC','META','MIDI','MIKA','MLPL','MNCN','PGAS','POWR','PPRO','PSAB','PTBA','PTPP','PWON','RALS','SCMA','SIDO','SMBR','SMGR','SMRA','SOCI','SRIL','SSIA','SSMS','TBIG','TCPI','TINS','TKIM','TLKM','TOWR','UNTR','UNVR','WEGE','WIKA','WOOD','WSBP','WSKT','WTON','ZINC','XXXX']
#kode=['BUMI','BWPT','CTRA','DGNS','DILD','DKFT','DMAS','DOID','DSNG','ELSA','ENRG','ERAA','ESSA','EXCL','FILM','FIRE','FPNI','GIAA','GJTL','GMFI','HKMU','HOKI','HRUM','ICBP','IKAN','IMAS','INCO','INDF','INKP','INPC','IPTV','ITMG','IRRA','ISAT','ISSP','JPFA','JSKY','JSMR','KAEF','KIJA','KIOS','KLBF','KOTA','KPIG','KRAS','LINK','LPKR','LPPF','LSIP','MAPI','MCOR','MDKA','MEDC','META','MIDI','MIKA','MLPL','MNCN','PGAS','POWR','PPRO','PSAB','PTBA','PTPP','PWON','RALS','SCMA','SIDO','SMBR','SMGR','SMRA','SOCI','SRIL','SSIA','SSMS','TBIG','TCPI','TINS','TKIM','TLKM','TOWR','UNTR','UNVR','WEGE','WIKA','WOOD','WSBP','WSKT','WTON','ZINC','AALI','ADMF','ADMG','AMOR','AMRT','ANDI','ARNA','ARTI','ARTO','ASGR','ASPI','AUTO','BABP','BAPI','BEEF','BELL','BGTG','BHAT','BHIT','BINA','BIPP','BIRD','BKSL','BLTA','BSIM','BTPS','CAMP','CARE','CASH','CASS','CBMF','CENT','CLEO','CMNP','COCO','CPIN','CSRA','DEAL','DIVA','DLTA','DMMX','DPNS','DSFI','DUCK','DWGL','DYAN','EMTK','FMII','FREN','GDST','GGRM','GOOD','GPRA','GSMF','HEAL','HEXA','HMSP','HOMI','HRTA','IMJS','IMPC','INAF','INDR','INDY','INTP','IPCC','IPCM','IPOL','ITIC','ITMA','JAST','JAYA','JKON','JRPT','JTPE','KAYU','KBLI','KEEN','KICI','KMTR','KPAS','KREN','LPCK','LUCK','MAIN','MAPA','MARI','MARK','MAYA','MCAS','MDIA','MEGA','MGRO','MLIA','MMLP','MPMX','MPPA','MRAT','MSIN','MTDL','MTPS','MYOR','NATO','NICK','NIKL','NISP','NOBU','OPMS','PALM','PAMG','PBID','PBRX','PLAN','PMMP','PNBN','PNBS','PNLF','POLL','PPGL','PPRE','PRDA','PRIM','PSSI','PTPW','PTRO','PTSN','PURA','PURE','PYFA','RAJA','REAL','ROTI','SAME','SBAT','SDRA','SFAN','SGER','SGRO','SHIP','SILO','SIMP','SIPD','SKRN','SMCB','SMDR','SMSM','SOTS','SPTO','SQMI','SRTG','TBLA','TDPM','TFAS','TGRA','TIRA','TMPO','TNCA','TOTL','TOYS','TPIA','TRIM','TRIN','TRIS','TRUK','TSPC','UCID','UFOE','ULTJ','UNIQ','VICI','VIVA','VRNA','WIFI','WIIM','WINS','WMUU','WOWS','XSSI','YELO','YULE','ZBRA','XXXX']
#kode=[ "AALI","TCPI","BMRI", "ASSA", "ADRO", "MDKA", "KLBF","AMRT","MNCN", "DOID", "SCMA","ABBA","MLPL","BRMS","BUMI",'XXXX']
#kode=['KAYU','KBLI','KEEN','KICI','KMTR','KPAS','KREN','LPCK','LUCK','MAIN','MAPA','MARI','MARK','MAYA','MCAS','MDIA','MEGA','MGRO','MLIA','MMLP','MPMX','MPPA','MRAT','MSIN','MTDL','MTPS','MYOR','NATO','NICK','NIKL','NISP','NOBU','OPMS','PALM','PAMG','PBID','PBRX','PLAN','PMMP','PNBN','PNBS','PNLF','POLL','PPGL','PPRE','PRDA','PRIM','PSSI','PTPW','PTRO','PTSN','PURA','PURE','PYFA','RAJA','REAL','ROTI','SAME','SBAT','SDRA','SFAN','SGER','SGRO','SHIP','SILO','SIMP','SIPD','SKRN','SMCB','SMDR','SMSM','SOTS','SPTO','SQMI','SRTG','TBLA','TDPM','TFAS','TGRA','TIRA','TMPO','TNCA','TOTL','TOYS','TPIA','TRIM','TRIN','TRIS','TRUK','TSPC','UCID','UFOE','ULTJ','UNIQ','VICI','VIVA','VRNA','WIFI','WIIM','WINS','WMUU','WOWS','XSSI','YELO','YULE','ZBRA','XXXX']

kode=[      "ADRO",
            "ANTM",
            "BRIS",
            "BRPT",
            "BUKA",
            "CPIN",
            "EMTK",
            "ERAA",
            "EXCL",
            "ICBP",
            "INCO",
            "INDF",
            "INKP",
            "INTP",
            "ITMG",
            "JPFA",
            "KLBF",
            "MIKA",
            "MNCN",
            "PGAS",
            "PTBA",
            "PTPP",
            "SMGR",
            "TINS",
            "TKIM",
            "TLKM",
            "TPIA",
            "UNTR",
            "UNVR",
            "WIKA",
            'XXXX'] # 'KAYU' diganti dg saham2 yg akan diambil, spt contoh di atasnya, 'XXXX' jangan dibuang
i=0
leng=len(kode)
#per=["120","240","D"]
#wak=[1000,480,240]

#per=["1","5","15","30","60","120","240","D","W"]
#per=["30"]
#wak=[5000,5000,5000,5000,5000,5000]
#wak=[2500,750,500,300,100,50,35,15,5]
#wak=[5000]  #4 minggu 
#wak=[1100]  #4 hk minggu 
#wak=[1400]  # 1 mgg/ 5 hk = 1600
#per=["120"]
#wak=[750]  #[15 => 4150, 30 =>2480 , 60 : 1480,  120 : 750   ;  D: 250]
#wak=[1400]
wak=[5000]
#wak=[2500]  # 9HK
#wak=[5000]
#per=["1"]
per=["D"]
#wak=[2700]
j=0
for j in range(len (wak)):
  k=0
  while i<leng:
    if kode[k]=='XXXX':
      break
    saham='IDX:'+kode[k]
    ws = create_connection(
        'wss://data.tradingview.com/socket.io/websocket',headers=headers)

    session= generateSession()
    print("session generated {}".format(session))

    chart_session= generateChartSession()
    print("chart_session generated {}".format(chart_session))

    # Then send a message through the tunnel 
    sendMessage(ws, "set_auth_token", ["unauthorized_user_token"])
    sendMessage(ws, "chart_create_session", [chart_session, ""])
    sendMessage(ws, "quote_create_session", [session])
    sendMessage(ws,"quote_set_fields", [session,"ch","chp","current_session","description","local_description","language","exchange","fractional","is_tradable","lp","lp_time","minmov","minmove2","original_name","pricescale","pro_name","short_name","type","update_mode","volume","currency_code","rchp","rtc"])
    print("Mulai ambil ?122")
    sendMessage(ws, "quote_add_symbols",[session, saham, {"flags":['force_permission']}])
    sendMessage(ws, "quote_fast_symbols", [session,saham])

    #st='~m~140~m~{"m":"resolve_symbol","p":}'
    #p1, p2 = filter_raw_message(st)
    #print("Mulai ambil ?128")
    #sendMessage(ws, "resolve_symbol", [chart_session,"symbol_1","={\"symbol\":\"IDX:ADRO\",\"adjustment\":\"splits\",\"session\":\"extended\"}"])
    #sendMessage(ws, "resolve_symbol", [chart_session,"symbol_1","={\"symbol\":\"IDX:ADRO\",\"adjustment\":\"splits\",\"session\":\"extended\"}"])
    #input()
    #Error : sendMessage(ws, "resolve_symbol", [chart_session,"symbol_1","={"symbol":"IDX:ADRO","adjustment":"splits","session":"extended\}"])
    simbol='={\"symbol\":\"'+saham+'\",\"adjustment\":\"splits\",\"session\":\"extended\"}'
    print(simbol)
    #input()
    sendMessage(ws, "resolve_symbol", [chart_session,"symbol_1",simbol])
    sendMessage(ws, "create_series", [chart_session, "s1", "s1", "symbol_1", per[j], wak[j]]) #5 menit 5 data ( max 5000 )
    #sendMessage(ws, "create_study", [chart_session,"st4","st1","s1","ESD@tv-scripting-101!",{"text":"BNEhyMp2zcJFvntl+CdKjA==_DkJH8pNTUOoUT2BnMT6NHSuLIuKni9D9SDMm1UOm/vLtzAhPVypsvWlzDDenSfeyoFHLhX7G61HDlNHwqt/czTEwncKBDNi1b3fj26V54CkMKtrI21tXW7OQD/OSYxxd6SzPtFwiCVAoPbF2Y1lBIg/YE9nGDkr6jeDdPwF0d2bC+yN8lhBm03WYMOyrr6wFST+P/38BoSeZvMXI1Xfw84rnntV9+MDVxV8L19OE/0K/NBRvYpxgWMGCqH79/sHMrCsF6uOpIIgF8bEVQFGBKDSxbNa0nc+npqK5vPdHwvQuy5XuMnGIqsjR4sIMml2lJGi/XqzfU/L9Wj9xfuNNB2ty5PhxgzWiJU1Z1JTzsDsth2PyP29q8a91MQrmpZ9GwHnJdLjbzUv3vbOm9R4/u9K2lwhcBrqrLsj/VfVWMSBP","pineId":"TV_SPLITS","pineVersion":"8.0"}])


    # Printing all the result
    a=""
    waktu1=time.time()
    while True and k<leng:
      try:
        result = ws.recv()
        print(result)
        a=a+result
      except Exception as e:
        print(e)
        break
    #per=["1","5","15","30","60","120","240","D","M","W"]
    #wak=[2100,420,150,75,35,20,10,10,2,1]
    #i=0
    #for i in range(10):
    generate_csv(a,saham,per[j],wak[j])
    waktu=time.time()-waktu1
    print("Waktu :",waktu)
    ws.close()
    k=k+1

Streaming output truncated to the last 5000 lines.

{"i":1880,"v":[1438653600.0,2481.87564,2495.766735,2444.83272,2454.09345,10298108.0]}
{"saham":"WIKA","indeks":"1880","date":"1438653600.0","open":"2481.87564","high":"2495.766735","low":"2444.83272","close":"2454.09345","vol":"10298108.0"}

{"i":1881,"v":[1438740000.0,2477.245275,2509.65783,2454.09345,2481.87564,29596370.0]}
{"saham":"WIKA","indeks":"1881","date":"1438740000.0","open":"2477.245275","high":"2509.65783","low":"2454.09345","close":"2481.87564","vol":"29596370.0"}

{"i":1882,"v":[1438826400.0,2491.13637,2528.17929,2472.61491,2481.87564,24767809.0]}
{"saham":"WIKA","indeks":"1882","date":"1438826400.0","open":"2491.13637","high":"2528.17929","low":"2472.61491","close":"2481.87564","vol":"24767809.0"}

{"i":1883,"v":[1438912800.0,2472.61491,2500.3971,2454.09345,2472.61491,14252008.0]}
{"saham":"WIKA","indeks":"1883","date":"1438912800.0","open":"2472.61491","high":"2500.3971","low":"2454.09345","close":"2472.61491","vol":"